In [2]:
using Oceananigans 
using Oceananigans.Units
using Oceananigans.TurbulenceClosures
using Oceananigans.Architectures: GPU
using CUDA: CuArray
using CairoMakie 
using NCDatasets
using Printf

In [3]:
# Model parameters
Nx = 300
Nz = 80
f = 1e-4               # Coriolis frequency [s⁻¹]
L_front = 10kilometers  # Initial front width [m]
aspect_ratio = 100      # L_front/H
Ro = 0.1           # Rossby number (defines M^2)

# Derived parameters
H_front = L_front/aspect_ratio
M² = (Ro^2*f^2*L_front)/H_front
Δb = M²*L_front
κh = 1e-4
κv = 1e-4

filename = "periodic_geostrophic_adjustment"

"periodic_geostrophic_adjustment"

In [4]:

grid = RectilinearGrid(GPU(),size = (Nx, Nz), 
                       x = (-L_front/2, L_front/2),
                       z = (-H_front, 0),
                       topology = (Periodic, Flat, Bounded))

300×1×80 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CUDAGPU with 3×0×3 halo
├── Periodic x ∈ [-5000.0, 5000.0) regularly spaced with Δx=33.3333
├── Flat y                         
└── Bounded  z ∈ [-100.0, 0.0]     regularly spaced with Δz=1.25

In [5]:
horizontal_closure = HorizontalScalarDiffusivity(ν=κh, κ=κh )
vertical_closure = VerticalScalarDiffusivity(ν=κv , κ=κv )
closure = (horizontal_closure, vertical_closure)

(HorizontalScalarDiffusivity{ExplicitTimeDiscretization}(ν=0.0001, κ=0.0001), VerticalScalarDiffusivity{ExplicitTimeDiscretization}(ν=0.0001, κ=0.0001))

In [6]:
model =  NonhydrostaticModel(; grid,
                coriolis = FPlane(f = f),
                buoyancy = BuoyancyTracer(),
                tracers = (:b,:T),
                advection = WENO(),
                closure = closure)

NonhydrostaticModel{GPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 300×1×80 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CUDAGPU with 3×0×3 halo
├── timestepper: RungeKutta3TimeStepper
├── advection scheme: WENO{3, Float64, Float32}(order=5)
├── tracers: (b, T)
├── closure: Tuple with 2 closures:
│   ├── HorizontalScalarDiffusivity{ExplicitTimeDiscretization}(ν=0.0001, κ=(b=0.0001, T=0.0001))
│   └── VerticalScalarDiffusivity{ExplicitTimeDiscretization}(ν=0.0001, κ=(b=0.0001, T=0.0001))
├── buoyancy: BuoyancyTracer with ĝ = NegativeZDirection()
└── coriolis: FPlane{Float64}(f=0.0001)

In [7]:
bᵢ(x, z) = Δb*sin(2*pi/L_front * x)
Tᵢ(x, z) = exp(-(x/(L_front/50)).^2)

set!(model, b= bᵢ, T= Tᵢ, u = 0, v = 0, w = 0)  # Start from rest

In [8]:
simulation = Simulation(model, Δt=20minutes, stop_time=4days)


Simulation of NonhydrostaticModel{GPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 20 minutes
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN days
├── Stop time: 4 days
├── Stop iteration: Inf
├── Wall time limit: Inf
├── Minimum relative step: 0.0
├── Callbacks: OrderedDict with 4 entries:
│   ├── stop_time_exceeded => 4
│   ├── stop_iteration_exceeded => -
│   ├── wall_time_limit_exceeded => e
│   └── nan_checker => }
├── Output writers: OrderedDict with no entries
└── Diagnostics: OrderedDict with no entries

In [9]:
conjure_time_step_wizard!(simulation, IterationInterval(20), cfl=0.2, max_Δt=20minutes)

In [10]:

wall_clock = Ref(time_ns())

function print_progress(sim)
    u, v, w = model.velocities
    progress = 100 * (time(sim) / sim.stop_time)
    elapsed = (time_ns() - wall_clock[]) / 1e9

    @printf("[%05.2f%%] i: %d, t: %s, wall time: %s, max(u): (%6.3e, %6.3e, %6.3e) m/s, next Δt: %s\n",
            progress, iteration(sim), prettytime(sim), prettytime(elapsed),
            maximum(abs, u), maximum(abs, v), maximum(abs, w), prettytime(sim.Δt))

    wall_clock[] = time_ns()

    return nothing
end

add_callback!(simulation, print_progress, IterationInterval(50))

In [113]:
# Output setup
u, v, w = model.velocities
b = model.tracers.b
wc = Field(@at (Center, Center, Center) model.velocities.w)
T = model.tracers.T

#For Lagrangian filtering
simulation.output_writers[:jld2fields] = JLD2Writer(
    model, (; b, u, v, w, wc,T), filename=filename * ".jld2", schedule=TimeInterval(1hour), overwrite_existing=true)


#for python visualisation
rm(filename * ".nc",force=true)
simulation.output_writers[:ncfields] = NetCDFWriter(
    model, (; b, u, v, w, wc,T), filename=filename * ".nc", schedule=TimeInterval(1hour), overwrite_existing=true)
    

NetCDFWriter scheduled on TimeInterval(1 hour):
├── filepath: periodic_geostrophic_adjustment_kh1e-2.nc
├── dimensions: time(0), x_faa(300), x_caa(300), z_aaf(81), z_aac(80)
├── 6 outputs: (v, w, T, wc, b, u)
├── array_type: Array{Float32}
├── file_splitting: NoFileSplitting
└── file size: 27.5 KiB

In [11]:
@info "Running the simulation..."

run!(simulation)

@info "Simulation completed in " * prettytime(simulation.run_wall_time)

[ Info: Running the simulation...
[ Info: Initializing simulation...


[00.00%] i: 0, t: 0 seconds, wall time: 13.552 seconds, max(u): (0.000e+00, 0.000e+00, 0.000e+00) m/s, next Δt: 20 minutes


[ Info:     ... simulation initialization complete (14.920 seconds)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (2.677 seconds).


[11.93%] i: 50, t: 11.457 hours, wall time: 3.032 seconds, max(u): (2.387e-02, 4.831e-02, 4.446e-04) m/s, next Δt: 8.744 minutes
[16.41%] i: 100, t: 15.757 hours, wall time: 147.242 ms, max(u): (1.809e-02, 7.771e-03, 2.844e-04) m/s, next Δt: 4.572 minutes
[20.71%] i: 150, t: 19.880 hours, wall time: 120.810 ms, max(u): (2.225e-02, 1.146e-02, 4.196e-04) m/s, next Δt: 5.532 minutes
[24.40%] i: 200, t: 23.424 hours, wall time: 128.232 ms, max(u): (2.505e-02, 4.579e-02, 4.867e-04) m/s, next Δt: 4.178 minutes
[28.32%] i: 250, t: 1.133 days, wall time: 126.036 ms, max(u): (1.102e-02, 5.823e-02, 2.150e-04) m/s, next Δt: 5.055 minutes
[32.56%] i: 300, t: 1.302 days, wall time: 127.570 ms, max(u): (2.763e-02, 2.642e-02, 6.022e-04) m/s, next Δt: 4.020 minutes
[36.33%] i: 350, t: 1.453 days, wall time: 131.918 ms, max(u): (5.080e-03, 2.644e-03, 1.221e-04) m/s, next Δt: 4.864 minutes
[40.78%] i: 400, t: 1.631 days, wall time: 122.973 ms, max(u): (2.821e-02, 2.802e-02, 6.456e-04) m/s, next Δt: 3.93

[ Info: Simulation is stopping after running for 21.003 seconds.
[ Info: Simulation time 4 days equals or exceeds stop time 4 days.
[ Info: Simulation completed in 21.006 seconds
